# **DESCRIPTA.AI** -> Extension Tables Processor: (tab.py)

tab.py is the central processing engine for the ZIP pipeline. It extracts LaTeX tables, generates three-stage natural language descriptions using an LLM (via Ollama), classifies graphability, restructures data for visualizations, and exports all results in multiple formats: JSON, DOCX, XLSX, and updated LaTeX. It also enriches descriptions using sentence embeddings with SentenceTransformer (MiniLM-L6).

## Functional Components


1. Initialization and Configuration

* Loads required NLTK tokenizers (punkt).

* Instantiates SentenceTransformer('all-MiniLM-L6-v2') for RAG.

* Global flags:

conversation_history: Maintains LLM session state.

LLM_AVAILABLE: Fallback switch for errors.

2. ZIP and File Handling

* create_unique_extraction_folder(...)

Generates a unique output directory for ZIP contents based on filename hash.

* extract_files_from_zip(...)

Unzips .tex files into the workspace.

* find_tex_files(...)

Recursively finds all .tex files in a directory.



3. Text Extraction and Preprocessing

* extract_tables_from_tex(...)

Regex extraction of \begin{table}...\end{table} blocks.

* extract_text_from_tex(...)

Cleans .tex documents to extract natural text, skipping LaTeX commands.

* fragment_text_to_sentences(...)

Tokenizes into sentences using NLTK.

* generate_embeddings(...)

Creates sentence embeddings for context matching (RAG).

* extract_title_and_sections(...)

Parses \title{} and \section{} metadata for document-level context.

4. LLM Interaction

Low-Level Helpers

* generate_text_adv(prompt)

Sends prompt to the LLM with history, updates conversation_history, handles errors.

* generate_text_rag_custom(prompt, sentences, embeddings)

RAG-based enrichment using most relevant sentence.

System Prompt Structure

Each LLM phase (initial, RAG, fusion, chart type classification) resets the conversation_history to a clean context:

In [ ]:
conversation_history = [{"role": "system", "content": "You are an assistant that..." }]

5. Table Classification and Description Pipeline

* has_enough_numeric_data(...)

Determines if a table is likely to be graphable (numeric density).

* classify_table_graphical(...)

Flags graphability based on numeric content.

* classify_table_chart_type(...)

Asks LLM to choose chart type (line, bar, pie, scatter).

* describe_table(...)

Generates initial LLM description with custom context.

* describe_initial(...)

Full phase 1: selects best sentence (via cosine similarity) and generates initial description.

* describe_rag_phase(...)

Full phase 2: LLM improves initial description using RAG.

* describe_final(...)

Full phase 3: merges phase 1 + 2 into a refined final description.

* process_single_table(...)

Applies all three phases to a single table with proper resets.

6. Output Generation

* save_details_to_json(...)

Dumps description metadata to JSON (tables: [ {...}, {...} ]).

* save_details_to_word(...)

Creates DOCX with 5-row table layout per extracted table:

Table #, LaTeX, Initial, RAG, Final descriptions.

* partition_and_save(...)

Splits tables into graphical/non-graphical and exports each group separately.

7. Visualization-Oriented Table Restructuring

* restructure_table_for_chart(...)

Converts a LaTeX-style table into Markdown layout suitable for visualization tools, following chart-specific templates.

* validate_markdown_table(...)

Ensures that reformatted tables meet structural integrity (header + numeric rows).

* markdown_to_dataframe(...)

Converts Markdown-formatted tables into pandas.DataFrame.

8. Excel Export for Graphable Tables

* save_restructured_tables_to_excel(...)

Saves all restructured graphable tables to individual Excel sheets, named T{number}_{chart_type} (31-char max).

9. Pipeline Entrypoint

* process_zip_and_generate_files(...)

Master orchestration function invoked by main.py:

Extracts .tex content.

Processes each table through 3 LLM phases.

Classifies graphability and chart type.

Generates full exports: JSON, DOCX, XLSX.

Logs all operations and saves ZIP for download.

In [ ]:
import os
import zipfile
import logging
import re
import json
import nltk
import numpy as np
from typing import List, Tuple, Dict, Any
from docx import Document
from docx.shared import Inches
from sentence_transformers import SentenceTransformer, util
from docx.document import Document as DocxDocument
from pylatexenc.latexencode import unicode_to_latex
import pandas as pd
from pandas import ExcelWriter
from pandas.io.excel._xlsxwriter import XlsxWriter
from io import StringIO
from pandas.errors import EmptyDataError

from pathlib import Path
import shutil
from docx.oxml      import OxmlElement
from docx.oxml.ns   import qn
from docx.shared    import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
import ollama



# -------------------------------------------------------------------
# Configuración inicial
# -------------------------------------------------------------------
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

for resource in ['punkt']:
    try:
        nltk.data.find(f'tokenizers/{resource}')
    except LookupError:
        nltk.download(resource)

logging.info("Using pytorch device: cpu")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

conversation_history: List[Dict[str, str]] = []
LLM_AVAILABLE = True

# -------------------------------------------------------------------
# Utility Functions
# -------------------------------------------------------------------
def create_unique_extraction_folder(zip_path: str) -> str:
    base_name = os.path.basename(zip_path)
    name_without_extension = os.path.splitext(base_name)[0]
    words = re.split(r'\W+', name_without_extension)
    folder_name = "extracted_tables_" + "_".join(words[:4])
    folder_path = os.path.join(os.getcwd(), folder_name)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

def extract_files_from_zip(zip_path: str, extract_to: str) -> None:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    logging.info(f"Extracted {zip_path} → {extract_to}")

def find_tex_files(extract_to: str) -> List[str]:
    tex_files = []
    for root, _, files in os.walk(extract_to):
        for file in files:
            if file.lower().endswith('.tex'):
                tex_files.append(os.path.join(root, file))
    return tex_files



def extract_tables_from_tex(tex_files: List[str]) -> List[str]:
    table_blocks = []
    table_pattern = re.compile(r'\\begin\{table\}(\[.*?\])?(.*?)\\end\{table\}', re.DOTALL)
    for tex_file in tex_files:
        with open(tex_file, 'r', encoding='utf-8') as f:
            content = f.read()
            blocks = table_pattern.findall(content)
            table_blocks.extend([b for (_, b) in blocks])
    logging.info(f"Extracted {len(table_blocks)} table blocks")
    return table_blocks

def extract_text_from_tex(tex_files: List[str]) -> str:
    plain_text_list = []
    for tex_file in tex_files:
        with open(tex_file, 'r', encoding='utf-8') as f:
            content = f.read()
            cleaned = re.sub(r'\\(?!title|section)[a-zA-Z]+\{.*?\}', '', content)
            plain_text_list.append(cleaned)
    plain_text = "\n".join(plain_text_list)
    logging.info("Extracted plain text from .tex files.")
    return plain_text

def fragment_text_to_sentences(text: str) -> List[str]:
    sentences = nltk.sent_tokenize(text)
    logging.info(f"Fragmented text into {len(sentences)} sentences.")
    return sentences

def generate_embeddings(sentences: List[str]) -> np.ndarray:
    embeddings = embedding_model.encode(sentences)
    logging.info("Generated embeddings for sentences.")
    return embeddings

def extract_title_and_sections(tex_files: List[str]) -> Tuple[str, List[Tuple[str, int]], str]:
    combined = ""
    for tex_file in tex_files:
        with open(tex_file, 'r', encoding='utf-8') as f:
            combined += f.read() + "\n"
    title_m = re.search(r'\\title\{([^}]*)\}', combined)
    title = title_m.group(1) if title_m else "No Title"
    sections = [(m.group(1), m.start()) for m in re.finditer(r'\\section\{([^}]*)\}', combined)]
    return title, sections, combined

# -------------------------------------------------------------------
# LLM and Text Generation
# -------------------------------------------------------------------
def trim_conversation_history(max_turns: int = 6) -> None:
    global conversation_history
    if len(conversation_history) > 2 * max_turns:
        conversation_history = conversation_history[-2 * max_turns:]

def generate_text_adv(prompt: str) -> str:
    global conversation_history, LLM_AVAILABLE

    if not LLM_AVAILABLE:
        return f"[LLM not available] {prompt}"
    if not prompt.strip():
        logging.warning("[LLM] Prompt vacío. Se omite el envío.")
        return "[Prompt vacío]"

    # Guardar prompt en log
    try:
        with open("prompt_history.log", "a", encoding="utf-8") as logf:
            logf.write(f"\n--- PROMPT ---\n{prompt}\n")
    except Exception as e:
        logging.warning(f"[LOG ERROR] No se pudo guardar prompt: {e}")

    try:
        conversation_history.append({"role": "user", "content": prompt})
        trim_conversation_history()

        response = ollama.chat(
            model="llama4:maverick",
            messages=conversation_history
        )

        result = response.get("message", {}).get("content", "")
        conversation_history.append({"role": "assistant", "content": result})

        return result.strip()

    except Exception as e:
        logging.error(f"[LLM ERROR] {e}")
        LLM_AVAILABLE = False
        return f"[Fallback - No LLM] {prompt}"

def generate_text_rag_custom(
    prompt: str,
    sentences: List[str],
    sentence_embeddings: np.ndarray
) -> str:

    if not sentences:
        return generate_text_adv(prompt)

    q_emb = embedding_model.encode(prompt)
    sims = util.cos_sim(q_emb, sentence_embeddings)[0].cpu().numpy()
    best = sentences[int(np.argmax(sims))]

    enriched = f"{prompt}\n\nContext:\n{best}"
    return generate_text_adv(enriched)

# -------------------------------------------------------------------
# Table Description Helpers
# -------------------------------------------------------------------
def has_enough_numeric_data(table_block: str, min_numeric_cells: int = 4) -> bool:

    lines = [l for l in table_block.splitlines() if l.strip()]
    if len(lines) < 2:
        return False

    # Count all numeric tokens
    nums = re.findall(r'\b[-+]?\d+(\.\d+)?\b', table_block)
    return len(nums) >= min_numeric_cells

def classify_table_graphical(table_block: str) -> bool:

    return has_enough_numeric_data(table_block)

# Then in your main loop, replace:

#    is_graph = classify_table_graphical(table_block)
#
# with simply:
#
#    is_graph = classify_table_graphical(block)

# And move the LLM chart‐type question out of this function:
def classify_table_chart_type(table_block: str) -> str:
    prompt = (
        "The table below contains numeric data. Choose exactly one chart type "
        "that best represents it: line, bar, pie, or scatter. "
        "If none of these is appropriate, reply 'other'.\n\n"
        f"{table_block}"
    )
    resp = generate_text_adv(prompt).strip().lower()
    logging.info(f"[ChartType] {resp}")
    for t in ('line','bar','pie','scatter'):
        if resp == t or t in resp:
            return t
    return 'other'



def get_best_matching_sentence(
    query: str,
    sentences: List[str],
    sentence_embeddings: np.ndarray
) -> str:

    q_emb = embedding_model.encode(query)
    sims = util.cos_sim(q_emb, sentence_embeddings)[0].cpu().numpy()
    return sentences[int(np.argmax(sims))]


def describe_table(table_block: str, context: str) -> Tuple[str, str]:
    petition = (
        "Describe the following table using only natural language. "
        "Do not use any LaTeX commands, formulas, or symbols. "
        "Instead, refer to cell positions with expressions like 'column 1, row 2'. "
        "Use the context below to enrich your interpretation.\n\n"
        f"Context:\n{context}\n\n"
        f"Table content (as reference only, do not use LaTeX syntax):\n{table_block}"
    )
    desc = generate_text_adv(petition)
    return desc, petition


def describe_initial(
    table_block: str,
    sentences: List[str],
    sentence_embeddings: np.ndarray
) -> Tuple[str, str, str]:

    best_ctx = get_best_matching_sentence(
        "Provide a detailed description of this table.",
        sentences,
        sentence_embeddings
    )
    init_desc, petition = describe_table(table_block, best_ctx)
    return init_desc, petition, best_ctx


def describe_rag_phase(
    initial_desc: str,
    sentences: List[str],
    sentence_embeddings: np.ndarray
) -> str:
    prompt = (
        "Improve the following table description using contextual information. "
        "Keep the response in English and in plain natural language. "
        "Do not include any LaTeX commands or symbols. "
        "Refer to the content using expressions like 'column X, row Y'.\n\n"
        f"Initial description:\n{initial_desc}"
    )
    return generate_text_rag_custom(prompt, sentences, sentence_embeddings)


def describe_final(initial_desc: str, rag_desc: str) -> str:
    prompt = (
        "Merge the two following descriptions into a single comprehensive final description. "
        "Use English only and plain natural language. "
        "Do not include LaTeX or mathematical symbols. "
        "Use expressions like 'column 2, row 3' to describe cell content.\n\n"
        f"Initial description:\n{initial_desc}\n\n"
        f"Contextual description:\n{rag_desc}"
    )
    return generate_text_adv(prompt)



def process_single_table(
    table_block: str,
    sentences: List[str],
    sentence_embeddings: np.ndarray,
    idx: int
) -> Dict[str, Any]:

    global conversation_history

    # 1) Reset inicial antes de cualquier fase
    conversation_history = [
        {
            "role": "system",
            "content": "Eres un asistente que describe tablas en lenguaje natural."
        }
    ]

    # 2) Clasificación local (sin LLM)
    is_graph = classify_table_graphical(table_block)

    # 3) Función helper para arrancar con historial limpio
    def fresh_phase(system_prompt: str, fn, *args, **kwargs):
        global conversation_history
        conversation_history = [
            {"role": "system", "content": system_prompt}
        ]
        return fn(*args, **kwargs)

    # Fase 1: descripción inicial
    init_desc, petition, best_ctx = fresh_phase(
        "Eres un asistente que crea la descripción inicial de una tabla.",
        describe_initial,
        table_block, sentences, sentence_embeddings
    )

    # Fase 2: RAG
    rag_desc = fresh_phase(
        "Eres un asistente que mejora la descripción usando contexto.",
        describe_rag_phase,
        init_desc, sentences, sentence_embeddings
    )

    # Fase 3: fusión final
    final_desc = fresh_phase(
        "Eres un asistente que fusiona dos descripciones en una final.",
        describe_final,
        init_desc, rag_desc
    )

    return {
        "table_number": idx + 1,
        "table_content": table_block,
        "graphically_representable": is_graph,
        "initial_description": init_desc,
        "rag_description": rag_desc,
        "final_description": final_desc,
        "best_matching_sentence": best_ctx
    }


# -------------------------------------------------------------------
# Output Saving Functions
# -------------------------------------------------------------------
def save_details_to_json(
    table_details: List[Dict[str, Any]],
    json_filename: str
) -> None:

    payload = {"tables": table_details}
    try:
        with open(json_filename, 'w', encoding='utf-8') as jf:
            json.dump(payload, jf, indent=4, ensure_ascii=False)
        logging.info(f"Saved JSON: {json_filename}")
    except Exception as e:
        logging.error(f"Error saving JSON {json_filename}: {e}")


def set_table_borders(table):
    tbl = table._tbl
    tblPr = tbl.tblPr or OxmlElement('w:tblPr')
    borders = OxmlElement('w:tblBorders')
    for name in ('top','left','bottom','right','insideH','insideV'):
        b = OxmlElement(f'w:{name}')
        b.set(qn('w:val'), 'single')
        b.set(qn('w:sz'), '8')    # 1pt
        b.set(qn('w:space'), '0')
        b.set(qn('w:color'), '000000')
        borders.append(b)
    tblPr.append(borders)
    if tbl.tblPr is None:
        tbl._element.append(tblPr)

def insert_page_margins(doc: DocxDocument,
                        top=0.5, bottom=0.5, left=0.5, right=0.5) -> None:
    sec = doc.sections[0]
    sec.top_margin = Inches(top)
    sec.bottom_margin = Inches(bottom)
    sec.left_margin = Inches(left)
    sec.right_margin = Inches(right)


def save_details_to_word(
    table_details: List[Dict[str, Any]],
    word_file: str,
    max_w: float = 4.5
) -> None:

    doc = Document()
    insert_page_margins(doc)

    for item in table_details:
        num   = item.get("table_number", "?")
        block = item.get("table_content", "").strip()
        d1    = item.get("initial_description", "")
        d2    = item.get("rag_description", "")
        d3    = item.get("final_description", "")

        # 5×2 table
        table = doc.add_table(rows=5, cols=2)
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        table.autofit   = True
        set_table_borders(table)

        # Row 1: merged title
        cell = table.cell(0,0)
        cell.merge(table.cell(0,1))
        p = cell.paragraphs[0]
        run = p.add_run(f"Table {num}")
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)

        # Row 2: merged LaTeX block
        cell = table.cell(1,0)
        cell.merge(table.cell(1,1))
        p = cell.paragraphs[0]
        p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        run = p.add_run(block)
        run.font.name = 'Courier New'
        run.font.size = Pt(9)

        # Row 3: Description 1
        lbl, val = table.cell(2,0), table.cell(2,1)
        p_lbl = lbl.paragraphs[0].add_run("Description 1:")
        p_lbl.font.name = 'Times New Roman'
        p_lbl.font.size = Pt(11)
        p_lbl.font.color.rgb = RGBColor(0,0,255)
        p_val = val.paragraphs[0].add_run(d1)
        p_val.font.name = 'Times New Roman'
        p_val.font.size = Pt(11)

        # Row 4: Description 2 (RAG)
        lbl, val = table.cell(3,0), table.cell(3,1)
        p_lbl = lbl.paragraphs[0].add_run("Description 2 (RAG):")
        p_lbl.font.name = 'Times New Roman'
        p_lbl.font.size = Pt(11)
        p_lbl.font.color.rgb = RGBColor(0,0,255)
        p_val = val.paragraphs[0].add_run(d2)
        p_val.font.name = 'Times New Roman'
        p_val.font.size = Pt(11)

        # Row 5: Description 3 (Final)
        lbl, val = table.cell(4,0), table.cell(4,1)
        p_lbl = lbl.paragraphs[0].add_run("Description 3 (Final):")
        p_lbl.font.name = 'Times New Roman'
        p_lbl.font.size = Pt(11)
        p_lbl.font.color.rgb = RGBColor(0,0,255)
        p_val = val.paragraphs[0].add_run(d3)
        p_val.font.name = 'Times New Roman'
        p_val.font.size = Pt(11)

        # Spacer & page break
        doc.add_paragraph()
        doc.add_page_break()

    try:
        doc.save(word_file)
        logging.info(f"Word saved → {word_file}")
    except Exception as e:
        logging.error(f"Error saving Word: {e}")


def partition_and_save(
    all_tables: List[Dict[str, Any]],
    out_graph: str,
    out_nongraph: str
) -> None:
    graph    = [t for t in all_tables if t["graphically_representable"]]
    nongraph = [t for t in all_tables if not t["graphically_representable"]]

    # correctly pass lists to JSON exporter
    save_details_to_json(graph,    "graphical_tables.json")
    save_details_to_json(nongraph, "nongraphical_tables.json")

    # lists to Word exporter
    save_details_to_word(graph,    out_graph)
    save_details_to_word(nongraph, out_nongraph)


# -------------------------------------------------------------------
# Main Processing
# -------------------------------------------------------------------


def classify_table_chart_type(table_block: str) -> str:

    global conversation_history

    # 1) Reset del historial
    conversation_history = [
        {
            "role": "system",
            "content": "Eres un asistente que elige el tipo de gráfico más adecuado."
        }
    ]

    # 2) Prompt de usuario
    prompt = (
        "La tabla siguiente tiene datos numéricos. "
        "Basado en sus valores y tendencias, responde con EXACTAMENTE uno de: "
        "`line`, `bar`, `pie` o `scatter`. "
        "No sugieras otro tipo.\n\n"
        f"{table_block}"
    )

    # 3) Llamada al LLM
    resp = generate_text_adv(prompt).strip().lower()

    # 4) Matching
    for t in ('line', 'bar', 'pie', 'scatter'):
        if resp == t or t in resp:
            return t
    return 'other'



# -------------------------------------------------------------------
# Reestructuración para el gráfico dado
# -------------------------------------------------------------------
def restructure_table_for_chart(
    table_block: str,
    chart_type: str,
    max_attempts: int = 2
) -> str:

    # Example snippets for each chart
    examples = {
        "line": (
            "Example format for a **line** chart:\n"
            "Time (s)\tSensor A\tSensor B\tSensor C\n"
            "0\t10\t20\t15\n"
            "1\t12\t22\t18\n"
            "2\t11\t19\t17\n\n"
            "Selection: A1:D4 (exclude the header row)."
        ),
        "bar": (
            "Example format for a **bar** chart:\n"
            "Category\tValue 1\tValue 2\tValue 3\n"
            "Group A\t30\t45\t25\n"
            "Group B\t20\t35\t40\n"
            "Group C\t50\t25\t30\n\n"
            "Note: Each row (e.g. Group A) represents a bar whose height"
            "is proportional to the sum of its values."
        ),
        "pie": (
            "Example format for a **pie** chart:\n"
            "Slice\tQ1\tQ2\tQ3\tQ4\n"
            "Year 2025\t10\t20\t30\t40\n\n"
            "Note: There is only one data row; the pie slices correspond to each column's "
            "percentage of the total."
        ),
        "scatter": (
            "Example format for a **scatter** chart:\n"
            "Time (s)\tHeight (m)\n"
            "0\t0.5\n"
            "1\t1.2\n"
            "2\t2.1\n"
            "3\t2.8"
        )
    }

    # Select the appropriate example (fallback to a generic template)
    example = examples.get(chart_type, (
        "Example format:\n"
        "Header1\tHeader2\n"
        "row1col1\trow1col2\n"
        "row2col1\trow2col2\n\n"
        "Use tabs to separate columns exactly."
    ))

    base_prompt = (
        f"You are a data formatting assistant. The user wants to plot this table as a **{chart_type}** chart. "
        "Rewrite the table in the exact Markdown format shown in the example below. "
        "First row = text headers; subsequent rows = numeric values; separate columns with tabs. Do not add aditional text.\n\n"
        f"{example}\n\n"
        "Now rewrite this table accordingly:\n"
        f"{table_block}"
    )

    prompt = base_prompt
    last_md = ""
    for attempt in range(1, max_attempts + 1):
        last_md = generate_text_adv(prompt).strip()
        if validate_markdown_table(last_md):
            return last_md
        # Reinforce formatting rules on retry
        prompt = (
            base_prompt +
            "\n\n⚠️ **Reminder:** The first row must be text headers; "
            "every following row must contain only numeric values. "
            "Separate columns with tabs exactly as in the example above."
        )

    return last_md



# -------------------------------------------------------------------
# Conversión Markdown → DataFrame
# -------------------------------------------------------------------

def validate_markdown_table(md_table: str) -> bool:

    lines = [l.strip() for l in md_table.splitlines() if l.strip()]
    if len(lines) < 2:
        return False

    # Header: ninguna celda es estrictamente numérica
    header_cells = re.split(r'\s+|\t|\|', lines[0])
    header_cells = [c for c in header_cells if c]
    if all(re.fullmatch(r'[-+]?\d+(\.\d+)?', cell) for cell in header_cells):
        return False

    # Filas de datos: cada celda debe ser numérica
    for row in lines[1:]:
        data_cells = re.split(r'\s+|\t|\|', row)
        data_cells = [c for c in data_cells if c]
        if not data_cells or not all(re.fullmatch(r'[-+]?\d+(\.\d+)?', cell) for cell in data_cells):
            return False

    return True



def markdown_to_dataframe(md_table: str) -> pd.DataFrame:

    # 1) Bail out on empty or whitespace-only input
    if not md_table or not md_table.strip():
        return pd.DataFrame()

    # 2) Tab-separated?
    first_line = md_table.splitlines()[0]
    if '\t' in first_line:
        try:
            return pd.read_csv(StringIO(md_table), sep='\t')
        except EmptyDataError:
            return pd.DataFrame()

    # 3) Pipe-style: keep lines containing '|' but ignore separator lines like |---|
    lines = [
        l.strip() for l in md_table.splitlines()
        if '|' in l and not re.fullmatch(r'\s*\|[-:\s]+\|\s*', l)
    ]

    # 4) If fewer than 2 lines, return empty DataFrame (or header-only)
    if len(lines) < 2:
        # Optional: parse header-only into columns
        header = [h.strip() for h in lines[0].split('|')] if lines else []
        return pd.DataFrame(columns=header)

    text = "\n".join(lines)
    try:
        df = pd.read_csv(
            StringIO(text),
            sep='|',
            engine='python',
            skipinitialspace=True
        )
        # Drop the first/last empty columns if they came from leading/trailing pipes
        if df.shape[1] > 2:
            return df.iloc[:, 1:-1]
        return df
    except EmptyDataError:
        return pd.DataFrame()


# -------------------------------------------------------------------
# Guardar todas las tablas reestructuradas en un Excel
# -------------------------------------------------------------------
def save_restructured_tables_to_excel(
    restructured: List[Dict[str, Any]],
    excel_path: str
) -> None:

    with XlsxWriter(excel_path) as writer:
        for item in restructured:
            idx   = item['table_number']
            ctype = item['chart_type']
            md    = item['markdown']

            try:
                df = markdown_to_dataframe(md)
            except EmptyDataError:
                df = pd.DataFrame()

            if df.empty:
                logging.warning("Skipping empty/invalid sheet T%d_%s", idx, ctype)
                continue

            sheet = f"T{idx}_{ctype}"[:31]
            df.to_excel(writer, sheet_name=sheet, index=False)

    logging.info("Saved restructured tables to %s", excel_path)



def process_zip_and_generate_files(
    zip_path: str,
    word_filename_graphical: str,
    word_filename_non_graphical: str
) -> None:

    # 1) Crear carpeta única y descomprimir
    extract_dir = create_unique_extraction_folder(zip_path)
    extract_files_from_zip(zip_path, extract_dir)
    tex_files = find_tex_files(extract_dir)

    # 2) Extraer contexto para RAG
    _, _, combined = extract_title_and_sections(tex_files)
    plain_text = extract_text_from_tex(tex_files)
    sentences = fragment_text_to_sentences(plain_text)
    sentence_embeddings = generate_embeddings(sentences)

    # 3) Procesar cada bloque de tabla
    all_details: List[Dict[str, Any]] = []
    restructured: List[Dict[str, Any]] = []

    for idx, block in enumerate(extract_tables_from_tex(tex_files)):
        logging.info(f"Processing Table {idx+1}")
        detail = process_single_table(block, sentences, sentence_embeddings, idx)

        if detail.get("graphically_representable", False):
            chart_type = classify_table_chart_type(detail["table_content"])
            md_table   = restructure_table_for_chart(detail["table_content"], chart_type)
            detail["chart_type"] = chart_type
            detail["restructured_markdown"] = md_table
            restructured.append({
                "table_number": detail["table_number"],
                "chart_type": chart_type,
                "markdown": md_table
            })

        all_details.append(detail)

    # 4) JSON + Word por graficabilidad
    partition_and_save(
        all_details,
        word_filename_graphical,
        word_filename_non_graphical
    )

    # 5) JSON + Word unificado
    save_details_to_json(all_details, "all_tables.json")
    save_details_to_word(all_details, "all_tables.docx")

    # 6) Excel con tablas graficables reestructuradas
    save_restructured_tables_to_excel(
        restructured,
        excel_path="restructured_tables.xlsx"
    )

    logging.info("Processing complete.")


# -------------------------------------------------------------------
# Entry Point
# -------------------------------------------------------------------
if __name__ == "__main__":
    process_zip_and_generate_files(
        "/Users/josu/Documents/TIC/LATEX/google.zip",
        "table_descriptions_graphical.docx",
        "table_descriptions_non_graphical.docx"
    )


## Outputs

* all_tables.json, graphical_tables.json, nongraphical_tables.json

* all_tables.docx, table_descriptions_graphical.docx, table_descriptions_non_graphical.docx

* restructured_tables.xlsx
